# Reconnaissance faciale via $k$ plus proches voisins ($k$-NN)

>## Partie 2: Reconnaissance faciale en temps réel

## Librairies

In [1]:
import pickle
import numpy as np

import cv2

from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

## Algorithme

In [2]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open('data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

Shape of visages matrix -->  (60, 50, 50, 3)


In [3]:
N = len(noms)
visages = visages.reshape(N, -1)

# Clustering K-means
kmeans = KMeans(n_clusters=3)  # Remplacez nombre_de_clusters par le nombre souhaité de clusters
kmeans.fit(visages)
labels = kmeans.labels_


## Exécution

In [4]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe

while True:
    
    ret, trame = camera.read()
    if ret == True:
        
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            
            visage = trame[y:y + h, x:x + l, :]
            visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
            
            texte = kmeans.predict(visage_redimensionne)
            
            cv2.putText(trame, str(texte[0]), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(trame, (x, y), (x + l, y + l), (0, 0, 255), 2)

        cv2.imshow('Reconnaissance faciale en temps réel', trame)
        
        if cv2.waitKey(1) == 27:
            break
            
    else:
        
        print("erreur")
        break

cv2.destroyAllWindows()
camera.release()

2024-02-29 15:02:44.165 Python[6906:7483694] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-02-29 15:02:45.636 Python[6906:7483694] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


**Exemple d'exécution du système...**

<img src="moi.png" width="400" height="auto" />